# ComplexChannelPatterns_Fig1f_RUN9
## 19 September 2023 - Fully checked (OK.)
### (C) Roeland C. van de Vijsel & Johan van de Koppel
### Royal Netherlands Institute for Sea Research (NIOZ), Department of Estuarine and Delta Systems, Yerseke, The Netherlands

#### This numerical model simulates saltmarsh development. The model output is used for the following manuscript:

##### van de Vijsel, R.C., van Belzen, J., Bouma, T.J., van der Wal, D., Borsje, B.W., Temmerman, S., Cornacchia, L., Gourgue, O., van de Koppel, J. (2023, Nature Communications). Vegetation controls on channel network complexity in coastal wetlands.

#### Whenever you use any part of this model code, please make sure to correctly refer to this manuscript and its authors.

#### IMPORTANT INFORMATION
Please note that in this model, X-coordinate increases from left to right and Y-coordinate increases from top to bottom.

However, in the manuscript, the Y-coordinate increases from left to right and X-coordinate increases from top to bottom.

Both in this model and in the manuscript, flow component u is in the X-direction and flow component v is in the Y-direction.

Hence, coordinates X and Y in this model script should be interpreted as Y and X, respectively, in the manuscript.

Idem, flow components u and v in this model should be interpreted as v and u, respectively, in the manuscript.

However, this reversal is done consistently throughout the model and the equations are implemented correctly.

Therefore, the model output is not affected.

##### This is the "default" model setup. It is used to generate the following figures in the manuscript:
- Fig. 1f
- Fig. 2d-f
- Supplementary Fig. 1b,d,f,h
- Supplementary Fig. 2e-h

### Model code

In [ ]:
## Set the run number and model name (OK.)

RUN = 9
ModelName = "ComplexChannelPatterns_Fig1f_RUN{}".format(RUN)
print(ModelName)

## Load Python packages (OK.)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import time,os
import scipy.io
%matplotlib inline
on = 1; off = 0

## Sensitivity analysis: vary parameters (e.g. PARa, PARb) over a certain range (OK.)

# PARa_arr = np.array([...,...,...])     # To be defined, e.g. Qq_arr = np.array([1e-2, 2e-2, 3e-2, 4e-2])
# PARb_arr = np.array([...,...,...])     # To be defined, e.g. Qs_arr = np.array([3e-4, 6e-4, 9e-4])

## Start the actual simulations, while looping over the range of parameters PARa and PARb (OK.)

for SUBRUNa in range(1):                 # for SUBRUNa in range(len(PARa_arr)):
    for SUBRUNb in range(1):             # for SUBRUNb in range(len(PARb_arr)):

        # PARa = PARa_arr[SUBRUNa]       # To be defined, e.g. Qq = Qq_arr[SUBRUNa]. Remember to disable PARa (in this example: Qq) below!
        # PARb = PARb_arr[SUBRUNb]       # To be defined, e.g. Qs = Qs_arr[SUBRUNb]. Remember to disable PARb (in this example: Qs) below!

        ## Simulation parameters (OK.)

        MORF    = 44712                  # [s/tide] Morphological acceleration factor, to speed up sedimentary and vegetation dynamics relative to hydrodynamics (1 hydrod sec = 1 geomorph M2-tide = 12.42h)
        facTY   = MORF/(60*60*24*365.25) # [yr/tide] Factor to translate number of tides into years

        ## Hydrodynamic parameters (OK.)

        H0      = 0.02 # 0.01                   # [m] Imposed initial water layer thickness
        g       = 9.81                   # [m/s^2] Gravitational acceleration constant
        HCrit   = 0.001                  # [m] Critical water layer thickness that always remains
        slope   = 0.000                  # [m/m] Uniform, subsediment bed slope
        DifU    = 5e-1                   # [m^2/s] Turbulent eddy viscosity
        rho     = 1000.0                 # [kg/m3] Reference water density

        ## Vegetation roughness parameters (OK.)

        nb      = 0.016                  # [s m^(-1/3)] Manning’s roughness coeff. for bare sediment
        nv      = 0.2                    # [s m^(-1/3)] Manning’s roughness coeff. for vegetated sediment
        Cb      = (H0**(1/6))/nb         # [m^(1/2)/s] Chezy coefficient of bare sediment (used as initial condition)

        ## Sedimentation parameters (OK.) 

        S_in    = MORF*5e-9              # [s/tide]*[m/s] Maximal sediment input rate
        Qs      = 6e-4                   # [m] Water layer thickness at which sediment input is halved
        E0      = MORF*2.5e-4 # MORF*3.5e-4            # [s/tide]*[s m^-2] Sediment erosion rate (in absence of vegetation and at unit bed shear stress
        pE      = 0.9                    # [-] Fraction by which sediment erosion is reduced when vegetation is at carrying capacity

        ## Vegetation parameters (OK.)       

        rr      = MORF*3.2e-8            # [s/tide]*[1/s] Intrinsic plant growth rate (= 1/year)
        kk      = 1500.0                 # [m^-2] Vegetation carrying capacity (max. stem density)
        Qq      = 2e-2                   # [m] Water layer thickness at which veg. growth is halved
        Ec      = MORF*1e-5              # [s/tide]*[s m^-2] Vegetation erosion rate (at unit bed shear stress)
        p_est   = 0.002                  # [-] Probability of vegetation seedling establishment

        ## Spatial gradient in D0 (OK.)

        Gradient_D0 = off
        D0      = MORF*1e-7              # [s/tide]*[m^2 s^-1] Sediment diffusivity in the absence of vegetation (if Gradient_D0 == off)
        D0min   = D0/2.0                 # [s/tide]*[m^2 s^-1] Idem, minimum value of D0 along the gradient (if Gradient_D0 == on)
        D0max   = D0*200.0               # [s/tide]*[m^2 s^-1] Idem, maximum value of D0 along the gradient (if Gradient_D0 == on)

        ## Spatial gradient in Hin (OK.)

        Gradient_Hin = off
        Hin     = 1.0e-5 # 1.4e-5                 # [m/s] Water input (“tidally averaged discharge velocity”) (if Gradient_Hin == off)
        HinMin  = 1.0e-5                 # [m/s] Idem, minimum value of Hin along the gradient (if Gradient_Hin == on)
        HinMax  = 1.0e-4                 # [m/s] Idem, maximum value of Hin along the gradient (if Gradient_Hin == on)

        ## Spatial gradient in DifD (OK.)

        Gradient_DifD = off
        DifD    = MORF*6e-9              # [s/tide]*[m2/s] Vegetation “diffusivity” (~ clonal expansion rate) (if Gradient_DifD == off)
        DifDmin = DifD/10.0              # [s/tide]*[m2/s] Idem, minimum value of DifD along the gradient (if Gradient_DifD == on)
        DifDmax = DifD*10.0              # [s/tide]*[m2/s] Idem, maximal value of DifD along the gradient (if Gradient_DifD == on)

        ## Spatial gradient in pD (OK.)

        Gradient_pD = off
        pD      = 0.99                   # [-] Fraction by which sediment diffusivity is reduced when vegetation is at carrying capacity (if Gradient_DifD == off)
        pDmin   = pD/10.0                # [-] Idem, minimum value of pD along the gradient (if Gradient_DifD == on)
        pDmax   = pD*10.0                # [-] Idem, maximum value of pD along the gradient (if Gradient_DifD == on)

        ## Infiltration rate (OK.)

        Df      = 0.000                  # [1/s] Vertical water drainage rate (or infiltration rate), equal to Hin/H0

        ## Time settings (OK.)   

        EndTime        = 70580           # [tide] End time (i.e., simulated time), in M2-tidal periods
        EndTimeYR      = EndTime*facTY   # [yr] End time (i.e., simulated time), in years
        NumFrames      = 100             # [-] Number of frames saved in the movie and output file
        dT             = 0.0125 #0.025           # [s] Time step size

        ## Construct the spatial grid (OK.)

        Block_Size_X   = 16              # [-] Thread block size in X-direction (use powers of 2)
        Block_Size_Y   = 16              # [-] Thread block size in Y-direction (use powers of 2)
        Block_Number_X = 128             # [-] Number of blocks in X-direction (use powers of 2)
        Block_Number_Y = 64              # [-] Number of blocks in Y-direction (use powers of 2)
        dX             = 0.5             # [m] Grid cell width
        dY             = 0.5             # [m] Grid cell height
        Grid_Width     = (Block_Size_X*Block_Number_X)     
        Grid_Height    = (Block_Size_Y*Block_Number_Y)
        LengthX        = Grid_Width*dX   # [m] Physical domain width
        LengthY        = Grid_Height*dY  # [m] Physical domain height
        print('Grid dimensions: %d x %d' % (Grid_Width,Grid_Height))

        ## Compute uniform initial conditions (OK.)

        h_uni     = H0                               # [m] Uniform water layer thickness
        v_uni     = (np.sqrt(slope)/nb)*h_uni**(2/3) # [m/s] Uniform flow, assuming balance between downslope gravitational acceleration and bed friction
        if v_uni == 0.0:                             # For zero slope, hence v_uni = 0, assume S_uni = 0
            S_uni = 0.0                              # [m] Uniform sediment layer thickness
        else:                                        # For non-zero slope, S_uni can be derived from the equation for dS/dt
            S_uni = (S_in/E0)*((h_uni-HCrit)*(h_uni**(1/3))/(Qs+h_uni-HCrit))/(g*nb*nb*v_uni*v_uni)

        ## Set up coordinate matrices (X: left-to-right = 0 to 1, Y: top-to-bottom = 0 to 1) (OK.)

        X, Y = np.meshgrid(np.linspace(0,1,Grid_Width), np.linspace(0,1,Grid_Height))

        ## Initialize the variable arrays (numpy arrays) (OK.)

        np.random.seed(2)                           # Generates random initial conditions

        u = np.zeros((Grid_Width*Grid_Height)) + 0.0                              # [m/s] Cross-slope flow
        v = np.zeros((Grid_Width*Grid_Height)) + v_uni                            # [m/s] Down-slope flow
        h = np.zeros((Grid_Width*Grid_Height)) + h_uni                            # [m] Water layer thickness
        b = np.zeros((Grid_Width*Grid_Height)) + (1.0-Y.flatten())*slope*LengthY; # [m] Uniform subsediment bedslope topography
        s = np.zeros((Grid_Width*Grid_Height)) + S_uni                            # [m] Sediment layer thickness
        d = (np.random.rand(Grid_Width*Grid_Height) < p_est) * kk                 # [m^-2] Vegetation stem density. Draw random numbers from uniform distribution [0,1]. Only fill those cells with number < p_est with vegetation, the others remain "bare"
        c = np.zeros((Grid_Width*Grid_Height)) + Cb                               # [m^(1/2)/s] Chezy coefficient
        t = np.zeros((Grid_Width*Grid_Height)) + (rho*g/Cb/Cb)*v_uni*v_uni        # [N/m2] Bed shear stress

        ## Convert the numpy arrays (64-bit) to 32-bit floats in order for the "host" (GPU) to handle them faster (OK.)

        u_host = u.astype(np.float32)
        v_host = v.astype(np.float32)
        h_host = h.astype(np.float32)
        b_host = b.astype(np.float32)
        s_host = s.astype(np.float32)
        d_host = d.astype(np.float32)
        c_host = c.astype(np.float32)
        t_host = t.astype(np.float32)

        import pyopencl as cl # Import openCL packages

        ## Define the device that is used (OK.)

        DeviceNr = 0   # 0 = GPU; 1 = Intel; 2 = AMD 
        os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'
        platform = cl.get_platforms()
        Devices  = platform[0].get_devices()
        context  = cl.Context([Devices[DeviceNr]])
        queue    = cl.CommandQueue(context)

        print(" Activated Compute Device: %s\n" % Devices[DeviceNr].name)

        mf       = cl.mem_flags # Set memory flags

        ## The array is allocated on the GPU and the initial values are copied onto it (OK.)

        u_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=u_host)
        v_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=v_host)
        h_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=h_host)
        b_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b_host)
        s_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=s_host)
        d_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=d_host)
        c_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c_host)
        t_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=t_host)

        ## Set up simulation parameters (OK.)

        global_size = u_host.shape
        local_size  = (Block_Number_X*Block_Number_Y,)

        ## Setting up the parameters for the OpenCL kernel. Seperate with comma without spaces (OK.)

        PassVars="H0,g,HCrit,slope,DifU,rho,nb,nv,Cb,S_in,Qs,E0,pE,rr,kk,Qq,Ec," + \
                 "Gradient_D0,D0,D0min,D0max,Gradient_Hin,Hin,HinMin,HinMax," + \
                 "Gradient_DifD,DifD,DifDmin,DifDmax,Gradient_pD,pD,pDmin,pDmax,Df," + \
                 "dT,dX,dY,Grid_Width,Grid_Height"

        PassVals=eval(PassVars)
        PassVars=PassVars.split(',')
        Params=""

        for ii in range(len(PassVals)):
            Params = Params+"#define " + PassVars[ii] + " " + str(PassVals[ii]) + " \n"

        ## OpenCL simulation kernel (OK.)

        ComputeCode = """

        float difco(float, float, size_t);                      // Declare the sediment diffusivity
        float d2_dxy2_S(__global float*, __global float*);      // Declare the sediment diffusion term

        #define cl_max(A,B)	(A>B?A:B)                           // Define the maximum function
        #define cl_min(A,B)	(A<B?A:B)                           // Define the minimum function

        #define ON 1                                            // Define the "on" switch
        #define OFF 0                                           // Define the "off" switch

        // (OK.) ////////////////////////////////////////////////////////////////////////

        float difco(float d1, float d2, size_t column)          // Define the sediment diffusivity
        {
            float dm = (d1+d2)/2.0 ;                            // Interpolate biomass between grid points (needed to compute sediment diffusivity) 

            #if Gradient_pD == ON                               // If Gradient_pD is on, make pD dependent on the width coordinate
                float pDx = exp( (float)(log((float)pDmax)-log((float)pDmin))*(float)column/(float)Grid_Width+log((float)pDmin) );
            #else                                               // If Gradient_pD is off, keep pD constant
                float pDx = (float)pD;
            #endif

            #if Gradient_D0 == ON                               // If Gradient_D0 is on, make D0 dependent on the width coordinate
                float D0x = exp( (float)(log((float)D0max)-log((float)D0min))*(float)column/(float)Grid_Width+log((float)D0min) );
            #else                                               // If Gradient_D0 is off, keep D0 constant
                float D0x = (float)D0;
            #endif

            float retval = D0x*(1.0 - (float)pDx*dm/(float)kk); // Compute sediment diffusivity and return the value
            return retval;                                      
        }   

        // (OK.) ////////////////////////////////////////////////////////////////////////

        float d2_dxy2_S(__global float* s, __global float* d)   // Define the sediment diffusion term
        {
            const float dx = dX;  // Forcing dX to become a float
            const float dy = dY;  // Forcing dY to become a float

            // Determine the position at which the current thread is computing
            const size_t current = get_global_id(0);            
            const size_t row     = (size_t)current/(size_t)Grid_Width; // Rounds down to integer, so row = 0 if current < Grid_Width, etc. 
            const size_t column  = (size_t)current%(size_t)Grid_Width;

            // Determine the positions neigboring the current position
            const size_t left    =  row      * Grid_Width + column - 1;
            const size_t right   =  row      * Grid_Width + column + 1;
            const size_t top     = (row - 1) * Grid_Width + column    ;
            const size_t bottom  = (row + 1) * Grid_Width + column    ;

            // Compute the actual sediment diffusion term and return its value
            return difco(d[right]  ,d[current], column)/dx/dx * (s[right]   - s[current]) -
                   difco(d[current],d[left]   , column)/dx/dx * (s[current] - s[left]) +
                   difco(d[bottom] ,d[current], column)/dy/dy * (s[bottom]  - s[current]) -
                   difco(d[current],d[top]    , column)/dy/dy * (s[current] - s[top]);
        }   

        // (OK.) ////////////////////////////////////////////////////////////////////////

        ////////////////////////////////////////////////////////////////////////////////
        // Simulation kernel
        ////////////////////////////////////////////////////////////////////////////////

        __kernel void SimulationKernel (__global float* u,
                                        __global float* v,
                                        __global float* h,
                                        __global float* b,
                                        __global float* s,
                                        __global float* d,
                                        __global float* c,
                                        __global float* t)
        {
            // A-priori variable definition
            float du, dv, dh, drh, ds;

            // Determine the position at which the current thread is computing
            const size_t current = get_global_id(0);            
            const size_t row     = (size_t)current/(size_t)Grid_Width; // Rounds down to integer, so row = 0 if current < Grid_Width, etc. 
            const size_t column  = (size_t)current%(size_t)Grid_Width;

            // --- Determine friction, and compute change in flow ----------------------

            // Net water flow speed [m/s]
            float uabs     = sqrt(u[current]*u[current]+v[current]*v[current]);

            // Thin film wetting-drying algoritm
            h[current]     = cl_max(h[current],HCrit);

            // Manning's bed roughness coefficient
            float nn = (float)nb + (float)(nv-nb)*(d[current]/(float)kk);

            // Chezy coefficient
            float Ct = (float)(1.0/(float)nn)*pow(h[current], (float)(1.0/6.0));

            // Evaluate the momentum equations
            if(row > 0 && row < Grid_Height-1 && column > 0 && column < Grid_Width-1)  // Away from the domain's boundaries
            {
                // Time derivative of water velocity u in the X direction
                du = - g*dO_dx(h,s,b)
                     - u[current]*d_dx(u)
                     - v[current]*d_dy(u)
                     - g/(Ct*Ct)*uabs*u[current]/h[current]
                     + DifU*d2_dxy2(u);

                // Time derivative of water velocity v in the Y direction
                dv = - g*dO_dy(h,s,b)
                     - u[current]*d_dx(v)
                     - v[current]*d_dy(v)
                     - g/(Ct*Ct)*uabs*v[current]/h[current]
                     + DifU*d2_dxy2(v);

                // Updating water flow
                u[current] = u[current] + du*dT;
                v[current] = v[current] + dv*dT;
            }
            else if(row==Grid_Height-1) // Bottom boundary
            {
                PersistentFluxBoundaries(u);  // Constant flux (free outflow) of u
                PersistentFluxBoundaries(v);  // Constant flux (free outflow) of v
            }
            else
            {
                ReflectingBoundaries(u,v);    // Other boundaries are a "solid wall"
            }

            // barrier(CLK_GLOBAL_MEM_FENCE);

            // --- Remaining state variables are computed here -----------------------------

            if(row > 0 && row < Grid_Height-1 && column > 0 && column < Grid_Width-1)  // Away from the domain's boundaries
            {
                // --- Continuity equation -------------------------------------------------
                dh = - d_uh_dx(u,h) - d_vh_dy(v,h);

                // Compute drainage term that is added to the continuity equation
                #if Gradient_Hin == ON
                    float HinX = exp( (float)(log((float)HinMax)-log((float)HinMin))*(float)column/(float)Grid_Width+log((float)HinMin) );
                    drh = HinX - Df*h[current];
                #else
                    drh = Hin - Df*h[current];
                #endif

                // --- Sediment equation -------------------------------------------

                float h_eff = h[current]-HCrit;   // Effective water layer thickness
                ds = S_in*h_eff/(Qs+h_eff) - E0*(1.0-pE*d[current]/kk)*s[current]*(g/(Ct*Ct))*uabs*uabs + d2_dxy2_S(s,d);

                // --- Vegetation equation -----------------------------------------

                #if Gradient_DifD == ON
                    float DifDx = exp( (float)(log((float)DifDmax)-log((float)DifDmin))*(float)column/(float)Grid_Width+log((float)DifDmin) );
                #else
                    float DifDx = DifD;
                #endif

                float dD = rr*d[current]*(1.0 - d[current]/kk)*Qq/(Qq+h_eff) - Ec*d[current]*(g/(Ct*Ct))*uabs*uabs + DifDx*d2_dxy2(d) ;

                // Updating water level, sediment height, and vegetation density
                h[current] = h[current] + dT*(dh+drh);
                s[current] = s[current] + dT*ds;
                d[current] = d[current] + dT*dD;
            }
            else if(row==Grid_Height-1) // Bottom boundary
            {
                NeumanBoundaries(h);         // Zero flux in h
                DirichletBoundaries(s,0.0);  // Fixed sediment height at the boundary
                NeumanBoundaries(d);         // Zero flux in d
            }
            else
            {
                NeumanBoundaries(h);   // Zero flux in h
                NeumanBoundaries(s);   // Zero flux in s
                NeumanBoundaries(d);   // Zero flux in d
            }

             c[current] = Ct;
             t[current] = (float)((rho*g/(Ct*Ct))*uabs*uabs);

        } // End SimulationKernel   

        """

        ## Build code (OK.)

        with open('HydroFunctions_iPy.cl', 'r') as myfile:
           SpatialFunctions = myfile.read()

        program = cl.Program(context, Params + SpatialFunctions + ComputeCode).build()

        ## Setting up a progress bar for the simulation (OK.)

        from ipywidgets import FloatProgress
        from IPython.display import display
        print("Progress :");
        PB = FloatProgress(min=0, max=NumFrames); display(PB) 

        ## Create arrays to store state variables as function of time (OK.)

        us = np.zeros((Grid_Height, Grid_Width, NumFrames))
        vs = np.zeros((Grid_Height, Grid_Width, NumFrames))
        hs = np.zeros((Grid_Height, Grid_Width, NumFrames))
        bs = np.zeros((Grid_Height, Grid_Width, NumFrames))
        ss = np.zeros((Grid_Height, Grid_Width, NumFrames))
        ds = np.zeros((Grid_Height, Grid_Width, NumFrames))
        cs = np.zeros((Grid_Height, Grid_Width, NumFrames))
        ts = np.zeros((Grid_Height, Grid_Width, NumFrames))

        ## Simulation loop (OK.)

        start_time = time.time() # Start a timer

        for count in range(NumFrames):

            # The simulation is executed <EndTime/dT/NumFrames> times (OK.)
            for jj in range(int(EndTime/dT/NumFrames)):
                program.SimulationKernel(queue, global_size, None, u_g, v_g, h_g, b_g, s_g, d_g, c_g, t_g)

            # Get the data from the GPU (OK.)
            cl.enqueue_copy(queue, u_host, u_g)
            cl.enqueue_copy(queue, v_host, v_g)
            cl.enqueue_copy(queue, h_host, h_g)
            cl.enqueue_copy(queue, b_host, b_g)
            cl.enqueue_copy(queue, s_host, s_g)
            cl.enqueue_copy(queue, d_host, d_g)
            cl.enqueue_copy(queue, c_host, c_g)
            cl.enqueue_copy(queue, t_host, t_g)

            # Store the state variables <NumFrames> times (OK.)
            us[:,:,count] = u_host.reshape(Grid_Height,Grid_Width)
            vs[:,:,count] = v_host.reshape(Grid_Height,Grid_Width)
            hs[:,:,count] = h_host.reshape(Grid_Height,Grid_Width)
            bs[:,:,count] = b_host.reshape(Grid_Height,Grid_Width)
            ss[:,:,count] = s_host.reshape(Grid_Height,Grid_Width)
            ds[:,:,count] = d_host.reshape(Grid_Height,Grid_Width)
            cs[:,:,count] = c_host.reshape(Grid_Height,Grid_Width)
            ts[:,:,count] = t_host.reshape(Grid_Height,Grid_Width)

            PB.value += 1 # Update the progress bar

        print(" Simulation took      : %1.1f (s)" % (time.time() - start_time))

        ## Plot end result (OK.)
        fs = 12 # fontsize

        if (True): 

            fig, ax = plt.subplots(2, 4, figsize=(16, 12))

            NetSpeed=np.sqrt(us[:,:,NumFrames-1]**2 + vs[:,:,NumFrames-1]**2)   # Absolute flow speed [m/s]

            # Sediment layer thickness
            im1=ax[0,0].imshow(ss[:,:,NumFrames-1], animated=True) #, interpolation='bilinear')
            ax[0,0].set_title('Sediment layer thickness [m]', fontsize=fs)
            ax[0,0].set_axis_off()
            f=fig.colorbar(im1, ax=ax[0,0], fraction=0.046, pad=0.04, orientation='horizontal')

            text1=fig.suptitle(ModelName+"_SUBRUNa{}_SUBRUNb{}       -       Time: %1.0f of %1.0f yr".format(SUBRUNa,SUBRUNb)  % (EndTime*facTY, EndTime*facTY), x=0.48, y=0.09, fontsize=fs+2);

            # Depth-averaged flow speed
            im2=ax[0,1].imshow(NetSpeed, animated=True)#, interpolation='bilinear')
            ax[0,1].set_title('Depth-averaged flow speed [m/s]', fontsize=fs)
            ax[0,1].set_axis_off()
            f=fig.colorbar(im2, ax=ax[0,1], fraction=0.046, pad=0.04, orientation='horizontal')

            # Vegetation stem density
            im3=ax[0,2].imshow(ds[:,:,NumFrames-1], animated=True)#, interpolation='bilinear')
            ax[0,2].set_title('Vegetation stem density [m^-2]', fontsize=fs)
            ax[0,2].set_axis_off()
            f=fig.colorbar(im3, ax=ax[0,2], fraction=0.046, pad=0.04, orientation='horizontal')

            im4=ax[0,3].imshow(bs[:,:,NumFrames-1], animated=True)#, interpolation='bilinear')
            ax[0,3].set_title('Subsediment bed topography [m]', fontsize=fs)
            ax[0,3].set_axis_off()
            f=fig.colorbar(im4, ax=ax[0,3], fraction=0.046, pad=0.04, orientation='horizontal')

            im5=ax[1,0].imshow(hs[:,:,NumFrames-1], animated=True)#, interpolation='bilinear')
            ax[1,0].set_title('Water layer thickness [m]', fontsize=fs)
            ax[1,0].set_axis_off()
            f=fig.colorbar(im5, ax=ax[1,0], fraction=0.046, pad=0.04, orientation='horizontal')

            im6=ax[1,1].imshow(cs[:,:,NumFrames-1], animated=True)#, interpolation='bilinear')
            ax[1,1].set_title('Chézy coefficient [m^(1/2)/s]', fontsize=fs)
            ax[1,1].set_axis_off()
            f=fig.colorbar(im6, ax=ax[1,1], fraction=0.046, pad=0.04, orientation='horizontal')

            im7=ax[1,2].imshow(ts[:,:,NumFrames-1], animated=True)#, interpolation='bilinear')
            ax[1,2].set_title('Bed shear stress [N/m2]', fontsize=fs)
            ax[1,2].set_axis_off()
            f=fig.colorbar(im7, ax=ax[1,2], fraction=0.046, pad=0.04, orientation='horizontal')

            im8=ax[1,3].imshow((ss[:,:,NumFrames-1]+hs[:,:,NumFrames-1]+bs[:,:,NumFrames-1]), animated=True)#, interpolation='bilinear')
            ax[1,3].set_title('Water surface elevation [m]', fontsize=fs)
            ax[1,3].set_axis_off()
            f=fig.colorbar(im8, ax=ax[1,3], fraction=0.046, pad=0.04, orientation='horizontal')

            fig.savefig('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}.png'.format(SUBRUNa,SUBRUNb),dpi = 500)

        ## Make animation (OK.)

        if (True):
            from matplotlib import animation, rc

            def updatefig(i): # To update the image at each iteration
                global us,vs,hs,bs,ss,ds,cs,ts
                NetSpeed=np.sqrt(us[:,:,i]**2 + vs[:,:,i]**2)
                im1.set_array(ss[:,:,i])
                im2.set_array(NetSpeed)
                im3.set_array(ds[:,:,i])
                im4.set_array(bs[:,:,i])
                im5.set_array(hs[:,:,i])
                im6.set_array(cs[:,:,i])
                im7.set_array(ts[:,:,i])
                im8.set_array((ss[:,:,i]+hs[:,:,i]+bs[:,:,i]))
                text1.set_text(ModelName+"_SUBRUNa{}_SUBRUNb{}       -       Time: %1.0f of %1.0f yr".format(SUBRUNa,SUBRUNb) % ((i+1)/NumFrames*EndTime*facTY, EndTime*facTY));
                return im1,im2,im3,im4,im5,im6,im7,im8

            ani = animation.FuncAnimation(fig, updatefig, interval=100, frames = NumFrames, repeat=False, blit=True)
            Writer = animation.writers['ffmpeg']
            writer = Writer(fps=10, metadata=dict(artist='Roeland C. van de Vijsel & Johan van de Koppel'), bitrate=1800)
            ani.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}.mp4'.format(SUBRUNa,SUBRUNb), writer=writer)

        ## Plot development of variables (mean/min/max) over time (OK.)

        mean_us = np.mean(us, axis=(0,1))
        mean_vs = np.mean(vs, axis=(0,1))
        mean_hs = np.mean(hs, axis=(0,1))
        mean_bs = np.mean(bs, axis=(0,1))
        mean_ss = np.mean(ss, axis=(0,1))
        mean_ds = np.mean(ds, axis=(0,1))
        mean_cs = np.mean(cs, axis=(0,1))
        mean_ts = np.mean(ts, axis=(0,1))

        min_us = np.min(us, axis=(0,1))
        min_vs = np.min(vs, axis=(0,1))
        min_hs = np.min(hs, axis=(0,1))
        min_bs = np.min(bs, axis=(0,1))
        min_ss = np.min(ss, axis=(0,1))
        min_ds = np.min(ds, axis=(0,1))
        min_cs = np.min(cs, axis=(0,1))
        min_ts = np.min(ts, axis=(0,1))

        max_us = np.max(us, axis=(0,1))
        max_vs = np.max(vs, axis=(0,1))
        max_hs = np.max(hs, axis=(0,1))
        max_bs = np.max(bs, axis=(0,1))
        max_ss = np.max(ss, axis=(0,1))
        max_ds = np.max(ds, axis=(0,1))
        max_cs = np.max(cs, axis=(0,1))
        max_ts = np.max(ts, axis=(0,1))

        tt = np.linspace(0,EndTime, num=NumFrames)*facTY

        fig, axs = plt.subplots(2, 4)
        axs[0, 0].plot(tt, mean_us, label ='mean u')
        axs[0, 0].plot(tt,  min_us, label = 'min u')
        axs[0, 0].plot(tt,  max_us, label = 'max u')
        axs[0, 0].get_xaxis().set_visible(False)
        axs[0, 0].set_title('u [m/s]')

        axs[0, 1].plot(tt, mean_vs, label ='mean v')
        axs[0, 1].plot(tt,  min_vs, label = 'min v')
        axs[0, 1].plot(tt,  max_vs, label = 'max v')
        axs[0, 1].get_xaxis().set_visible(False)
        axs[0, 1].set_title('v [m/s]')

        axs[0, 2].plot(tt, mean_hs, label ='mean h')
        axs[0, 2].plot(tt,  min_hs, label = 'min h')
        axs[0, 2].plot(tt,  max_hs, label = 'max h')
        axs[0, 2].get_xaxis().set_visible(False)
        axs[0, 2].set_title('h [m]')

        axs[0, 3].plot(tt, mean_bs, label ='mean b')
        axs[0, 3].plot(tt,  min_bs, label = 'min b')
        axs[0, 3].plot(tt,  max_bs, label = 'max b')
        axs[0, 3].get_xaxis().set_visible(False)
        axs[0, 3].set_title('b [m]')

        axs[1, 0].plot(tt, mean_ss, label ='mean s')
        axs[1, 0].plot(tt,  min_ss, label = 'min s')
        axs[1, 0].plot(tt,  max_ss, label = 'max s')
        axs[1, 0].set_title('s [m]')

        axs[1, 1].plot(tt, mean_ds, label ='mean d')
        axs[1, 1].plot(tt,  min_ds, label = 'min d')
        axs[1, 1].plot(tt,  max_ds, label = 'max d')
        axs[1, 1].set_title('d [m^-2]')

        axs[1, 2].plot(tt, mean_cs, label ='mean c')
        axs[1, 2].plot(tt,  min_cs, label = 'min c')
        axs[1, 2].plot(tt,  max_cs, label = 'max c')
        axs[1, 2].set_title('c [m^(1/2)/s]')

        axs[1, 3].plot(tt, mean_ts, label ='mean t')
        axs[1, 3].plot(tt,  min_ts, label = 'min t')
        axs[1, 3].plot(tt,  max_ts, label = 'max t')
        axs[1, 3].set_title('t [N/m2]')

        text2=fig.suptitle(ModelName+"_SUBRUNa{}_SUBRUNb{}".format(SUBRUNa,SUBRUNb), x=0.48, y=0.05);

        fig.savefig('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_overview.png'.format(SUBRUNa,SUBRUNb),dpi = 500)

        ## Save output in Python-format (OK.)

        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_u'.format(SUBRUNa,SUBRUNb),us[:,:,NumFrames-1])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_v'.format(SUBRUNa,SUBRUNb),vs[:,:,NumFrames-1])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_h'.format(SUBRUNa,SUBRUNb),hs[:,:,NumFrames-1])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_b'.format(SUBRUNa,SUBRUNb),bs[:,:,NumFrames-1])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_s'.format(SUBRUNa,SUBRUNb),ss[:,:,NumFrames-1])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_d'.format(SUBRUNa,SUBRUNb),ds[:,:,NumFrames-1])

        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_us'.format(SUBRUNa,SUBRUNb),us[:,:,:])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_vs'.format(SUBRUNa,SUBRUNb),vs[:,:,:])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_hs'.format(SUBRUNa,SUBRUNb),hs[:,:,:])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_bs'.format(SUBRUNa,SUBRUNb),bs[:,:,:])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_ss'.format(SUBRUNa,SUBRUNb),ss[:,:,:])
        np.save('RUN{}/'.format(RUN) + ModelName + '_SUBRUNa{}_SUBRUNb{}_ds'.format(SUBRUNa,SUBRUNb),ds[:,:,:])